<a href="https://colab.research.google.com/github/Rohan-Bha/Special-Topcis-MIS-IntroToAI2026/blob/main/Lab_3_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install openai -q
from openai import OpenAI
import os

client = OpenAI(api_key="API_Key")

def call_llm(system_prompt, user_message):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_message}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

In [4]:
customer_message = """
I was charged TWICE for my subscription this month and nobody is responding to my emails!
This is completely unacceptable. I need this fixed immediately or I'm disputing with my bank.
"""

In [6]:
STEP1_SYSTEM = """You are a customer support triage assistant.
Your ONLY job is to classify incoming support tickets.

Classify the message into:
- CATEGORY: one of [billing, technical, account_access, shipping, product_defect, other]
- SEVERITY: one of [low, medium, high, critical]
- SENTIMENT: one of [calm, frustrated, angry]

Respond ONLY in this exact format, no extra text:
CATEGORY: <category>
SEVERITY: <severity>
SENTIMENT: <sentiment>
REASON: <one sentence explaining your classification>"""

step1_output = call_llm(STEP1_SYSTEM, f'Customer message: {customer_message}')
print('=== STEP 1 OUTPUT (Classification) ===')
print(step1_output)

=== STEP 1 OUTPUT (Classification) ===
CATEGORY: billing  
SEVERITY: critical  
SENTIMENT: angry  
REASON: The customer is experiencing a double charge and is expressing urgency and frustration due to lack of response from support.


In [7]:
STEP2_SYSTEM = """You are a customer support intake specialist.
You receive a classified support ticket and must identify what information is still needed.

Rules:
- Ask for no more than 3 specific missing details.
- Be polite and empathetic, especially if sentiment is frustrated or angry.
- Do NOT try to solve the problem yet — only gather information.
- Keep your message under 80 words.
- Acknowledge the customer's frustration if sentiment is frustrated or angry.

Respond with:
MISSING_INFO: <comma-separated list of what you need>
CUSTOMER_REPLY: <the actual message you'd send the customer>"""

step2_input = f"""
Original customer message: {customer_message}

Classification results:
{step1_output}
"""

step2_output = call_llm(STEP2_SYSTEM, step2_input)
print('=== STEP 2 OUTPUT (Missing Info Request) ===')
print(step2_output)

=== STEP 2 OUTPUT (Missing Info Request) ===
MISSING_INFO: account email, transaction dates, subscription type  
CUSTOMER_REPLY: I understand how frustrating this situation is, and I’m sorry for the inconvenience. To assist you better, could you please provide the email associated with your account, the dates of the transactions, and the type of subscription you have? Thank you for your patience.


In [8]:
STEP3_SYSTEM = """You are a senior customer support specialist.
You receive a classified ticket and gathered context. Propose a concrete solution.

Guidelines:
- Provide 1 primary solution and 1 backup option.
- Specify the timeline for resolution.
- Maintain a professional and apologetic tone if severity is high or critical.
- Keep the customer reply under 120 words.
- Do NOT make promises you cannot keep.

Respond with:
PRIMARY_SOLUTION: <internal note>
BACKUP_OPTION: <fallback plan>
TIMELINE: <expected resolution time>
CUSTOMER_REPLY: <message to send the customer>"""

additional_info = 'Account email: user@example.com, Order #: ORD-2024-88721, charge dates: Nov 1 and Nov 2'

step3_input = f"""
Original message: {customer_message}
Classification: {step1_output}
Missing info identified: {step2_output}
Customer's additional information: {additional_info}
"""

step3_output = call_llm(STEP3_SYSTEM, step3_input)
print('=== STEP 3 OUTPUT (Proposed Solution) ===')
print(step3_output)

=== STEP 3 OUTPUT (Proposed Solution) ===
PRIMARY_SOLUTION: Investigate the double charge by reviewing the transaction history for the account user@example.com and initiate a refund for the duplicate charge. 

BACKUP_OPTION: If the refund process encounters delays, offer a credit to the customer's account as a temporary measure until the issue is resolved.

TIMELINE: I expect to have an update on the investigation and potential resolution within 48 hours.

CUSTOMER_REPLY: Thank you for your patience. I will investigate the double charge on your account (user@example.com) and initiate a refund for the duplicate charge. I expect to have an update for you within 48 hours. If there are any delays, I will offer a credit to your account as a temporary measure. I sincerely apologize for the inconvenience this has caused.


In [9]:
STEP4_SYSTEM = """You are a customer support manager responsible for escalation decisions.

Escalation rules:
- ESCALATE to Tier 2 if: severity is critical OR sentiment is angry AND unresolved after 48h.
- ESCALATE to Finance if: category is billing AND involves a duplicate charge.
- CLOSE if: solution was provided AND severity is low or medium.
- FLAG for REVIEW if: unclear resolution or conflicting signals.

Respond with:
DECISION: <ESCALATE_TIER2 | ESCALATE_FINANCE | CLOSE | FLAG_REVIEW>
ASSIGNED_TO: <team or person>
PRIORITY: <P1 | P2 | P3>
NOTES: <one sentence internal note>"""

step4_input = f"""
Customer message: {customer_message}
Step 1 - Classification: {step1_output}
Step 2 - Missing info: {step2_output}
Step 3 - Proposed solution: {step3_output}
"""

step4_output = call_llm(STEP4_SYSTEM, step4_input)
print('=== STEP 4 OUTPUT (Escalation Decision) ===')
print(step4_output)

=== STEP 4 OUTPUT (Escalation Decision) ===
DECISION: ESCALATE_FINANCE  
ASSIGNED_TO: Finance Team  
PRIORITY: P1  
NOTES: Customer is facing a critical billing issue with a duplicate charge and has expressed urgency.


In [10]:
print('=' * 60)
print('CUSTOMER SUPPORT CHAIN — FULL SUMMARY')
print('=' * 60)
print(f'\nCUSTOMER MESSAGE:\n{customer_message.strip()}')
print(f'\nSTEP 1 — CLASSIFICATION:\n{step1_output}')
print(f'\nSTEP 2 — MISSING INFO REQUEST:\n{step2_output}')
print(f'\nSTEP 3 — PROPOSED SOLUTION:\n{step3_output}')
print(f'\nSTEP 4 — ESCALATION DECISION:\n{step4_output}')
print('\n' + '=' * 60)
print('Chain complete. Each step used the prior step\'s output.')


CUSTOMER SUPPORT CHAIN — FULL SUMMARY

CUSTOMER MESSAGE:
I was charged TWICE for my subscription this month and nobody is responding to my emails!
This is completely unacceptable. I need this fixed immediately or I'm disputing with my bank.

STEP 1 — CLASSIFICATION:
CATEGORY: billing  
SEVERITY: critical  
SENTIMENT: angry  
REASON: The customer is experiencing a double charge and is expressing urgency and frustration due to lack of response from support.

STEP 2 — MISSING INFO REQUEST:
MISSING_INFO: account email, transaction dates, subscription type  
CUSTOMER_REPLY: I understand how frustrating this situation is, and I’m sorry for the inconvenience. To assist you better, could you please provide the email associated with your account, the dates of the transactions, and the type of subscription you have? Thank you for your patience.

STEP 3 — PROPOSED SOLUTION:
PRIMARY_SOLUTION: Investigate the double charge by reviewing the transaction history for the account user@example.com and in